## sales_data from the Walmart supplier list
## customer_data and store_data from Kaggle

In [4]:
import os
os.chdir('/Users/sridharmalladi')

In [5]:
#Loading all three datasets
import pandas as pd
customer_data = pd.read_csv("customer_data.csv") 
store_data = pd.read_csv("store_data.csv")
sales_data = pd.read_csv("sales_data.csv")

In [17]:
#Checking for the missing values in customer data
missing_values = customer_data.isnull().sum()
print("Missing values in customer_data:")
print(missing_values)
#Significant number of missing values to work with

Missing values in customer_data:
customer_id      0
age            215
gender         205
date             0
location       204
num_items      197
avg_price      223
dtype: int64


## Handling the Missing Values in the customer_data


In [ ]:
#Cleaning the dataset from variable to variable
import numpy as np
#Importing SimpleImputer for handling missing values
from sklearn.impute import SimpleImputer  

def clean_customer_data(df):
    #Creating a copy of the original dataframe to preserve the original data
    df_cleaned = df.copy()
    
    #Handling Missing Ages
    #Creating an imputer object that will replace missing values with the median
    #Using median instead of mean because it's more robust to outliers which can be present
    age_imputer = SimpleImputer(strategy='median')
    #fit_transform() function learns the median from the data and applies the imputation
    df_cleaned['age'] = age_imputer.fit_transform(df_cleaned[['age']])
    
    #Handling Missing Gender
    #For categorical data like gender, we use the mode
    df_cleaned['gender'] = df_cleaned['gender'].fillna(df_cleaned['gender'].mode()[0])
    
    #Handling Missing Location
    #Similar to gender, we use the most common location to fill missing values
    df_cleaned['location'] = df_cleaned['location'].fillna(df_cleaned['location'].mode()[0])
    
    #Handling Missing Number of Items
    #Using median because it's less sensitive to extreme values
    num_items_imputer = SimpleImputer(strategy='median')
    df_cleaned['num_items'] = num_items_imputer.fit_transform(df_cleaned[['num_items']])
    
    #Handling Missing Average Prices
    #Median is especially good for prices as they often have outliers
    price_imputer = SimpleImputer(strategy='median')
    df_cleaned['avg_price'] = price_imputer.fit_transform(df_cleaned[['avg_price']])
    
    # Return the cleaned dataset
    return df_cleaned

#Applying the cleaning function to your dataset
customer_data_cleaned = clean_customer_data(customer_data)

#Printing before and after summaries to verify the cleaning
print("\nOriginal dataset missing values:")
print(customer_data.isnull().sum())  
print("\nCleaned dataset missing values:")
print(customer_data_cleaned.isnull().sum())  
#Saving the cleaned dataset
customer_data_cleaned.to_csv('customer_data_cleaned.csv', index=False)



Original dataset missing values:
customer_id      0
age            215
gender         205
date             0
location       204
num_items      197
avg_price      223
dtype: int64

Cleaned dataset missing values:
customer_id    0
age            0
gender         0
date           0
location       0
num_items      0
avg_price      0
dtype: int64


In [20]:
#Checking for the missing values in customer data
missing_values = store_data.isnull().sum()
print("Missing values in store_data:")
print(missing_values)
#Since the value is negligible we can move forward with the dataset

Missing values in store_data:
store_id                 0
location                 0
state                    0
category                 0
size_sqft                0
mom_growth               1
yoy_growth               0
negative_equity          0
delinquency              0
inventory_turnover       0
employee_count           0
customer_satisfaction    1
dtype: int64


In [21]:
#Checking for the missing values in customer data
missing_values = sales_data.isnull().sum()
print("Missing values in sales_data:")
print(missing_values)
#no missing values found!

Missing values in sales_data:
transactionID     0
storeID           0
date              0
state             0
season            0
category          0
is_holiday        0
total_amount      0
profit_margin     0
items_count       0
payment_method    0
customer_type     0
dtype: int64


## Adding the Latitudes and longitudes to the store_data.csv

In [15]:
import pandas as pd
from geopy.geocoders import Nominatim
import time

#Reading store data
print("Loading store data...")
stores = pd.read_csv('store_data.csv')

#Initializing geocoder for location optimization 
geolocator = Nominatim(user_agent="retail_store_locator")

#Function to get coordinates with error handling perfectly
def get_coordinates(address):
    try:
        #Adding USA as that is where our data is from
        full_address = f"{address}, USA"
        location = geolocator.geocode(full_address)
        if location:
            return location.latitude, location.longitude
        return None, None
    except Exception as e:
        print(f"Error with address: {address}")
        print(f"Error message: {str(e)}")
        return None, None
    finally:
        #Managing the API limits
        time.sleep(1)

#Getting coordinates for each store
print("\nGetting coordinates...")
coordinates = []
for idx, address in enumerate(stores['location']):
    lat, lon = get_coordinates(address)
    coordinates.append((lat, lon))
    if (idx + 1) % 5 == 0:  # Progress update every 5 stores
        print(f"Processed {idx + 1} stores...")

#Adding new columns
stores['latitude'] = [coord[0] for coord in coordinates]
stores['longitude'] = [coord[1] for coord in coordinates]

#Saving updated data
stores.to_csv('store_data.csv', index=False)

#Results
print("\nResults:")
print(f"Total stores: {len(stores)}")
print(f"Coordinates found: {stores['latitude'].notna().sum()}")
print("\nSample of updated data:")
print(stores[['location', 'latitude', 'longitude']].head())

Loading store data...

Getting coordinates...
Processed 5 stores...
Processed 10 stores...
Processed 15 stores...
Processed 20 stores...
Processed 25 stores...
Processed 30 stores...
Error with address: 615 S Cumberland St, Lebanon, AL 37087
Error message: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=615+S+Cumberland+St%2C+Lebanon%2C+AL+37087%2C+USA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Processed 35 stores...
Processed 40 stores...
Processed 45 stores...
Processed 50 stores...
Processed 55 stores...
Processed 60 stores...

Results:
Total stores: 60
Coordinates found: 45

Sample of updated data:
                                       location   latitude   longitude
0     3826 S Suncoast Blvd, Homosassa, FL 34448        NaN         NaN
1         1936 N Lecanto Hwy, Lecanto, FL 34461  28.892694  -82.483937
2     68

## Looking for missing latitudes and longitudes

In [16]:
#Looking for missing latitudes and longitudes
import pandas as pd

# Read store data
stores = pd.read_csv('store_data.csv')

# Find missing locations
missing_coords = stores[stores['latitude'].isna() | stores['longitude'].isna()]

# Print missing locations
print("Stores missing coordinates:")
for idx, store in missing_coords.iterrows():
    print(f"{store['location']} ({store['state']})")

Stores missing coordinates:
3826 S Suncoast Blvd, Homosassa, FL 34448 (FL)
6885 S Suncoast Blvd, Homosassa, FL 34446 (FL)
2461 E Gulf To Lake Hwy, Inverness, FL 34453 (FL)
2800 NC-24 #87, Cameron, NC 28326 (NC)
1550 Nashville Rd, Franklin, KY 42134 (KY)
2100 88th St, North Bergen, NY 07047 (NY)
141 Washington Ave Ext, Albany, NY 12205 (NY)
300 Pleasant Grove Rd Ste 600, Mount Juliet, TN 37122 (TN)
304 S Rockwood Dr, Cabot, AR 72023 (AR)
615 S Cumberland St, Lebanon, AL 37087 (AL)
300 Pleasant Grove Rd Ste 600, Mount Juliet, IL 61522 (IL)
4424 Lebanon Pike, Hermitage, IL 37076 (IL)
5511 Murfreesboro Rd, La Vergne, IL 37086 (IL)
2000 Old Fort Pkwy, Murfreesboro, IL 37129 (IL)
200 Walmart Way, Avon, IN 46123 (IN)


In [6]:
import pandas as pd

# Read store data
stores = pd.read_csv('store_data.csv')

#Coordinates for missing locations
missing_coordinates = {
    "3826 S Suncoast Blvd, Homosassa, FL 34448": (28.7892, -82.5958),
    "6885 S Suncoast Blvd, Homosassa, FL 34446": (28.7666, -82.5752),
    "2461 E Gulf To Lake Hwy, Inverness, FL 34453": (28.8372, -82.3738),
    "2800 NC-24 #87, Cameron, NC 28326": (35.3276, -79.2534),
    "1550 Nashville Rd, Franklin, KY 42134": (36.7081, -86.5777),
    "2100 88th St, North Bergen, NY 07047": (40.8144, -74.0023),
    "141 Washington Ave Ext, Albany, NY 12205": (42.6901, -73.8490),
    "300 Pleasant Grove Rd Ste 600, Mount Juliet, TN 37122": (36.1991, -86.5186),
    "304 S Rockwood Dr, Cabot, AR 72023": (34.9744, -92.0167),
    "615 S Cumberland St, Lebanon, AL 37087": (33.5186, -86.8104),
    "300 Pleasant Grove Rd Ste 600, Mount Juliet, IL 61522": (41.8781, -87.6298),
    "4424 Lebanon Pike, Hermitage, IL 37076": (41.8781, -87.6298),
    "5511 Murfreesboro Rd, La Vergne, IL 37086": (41.8781, -87.6298),
    "2000 Old Fort Pkwy, Murfreesboro, IL 37129": (41.8781, -87.6298),
    "200 Walmart Way, Avon, IN 46123": (39.7684, -86.1581)
}

#Updating only missing coordinates
for location, (lat, lon) in missing_coordinates.items():
    mask = stores['location'] == location
    stores.loc[mask, 'latitude'] = lat
    stores.loc[mask, 'longitude'] = lon

#Saving updated data
stores.to_csv('store_data.csv', index=False)

#Verifying updates
still_missing = stores[stores['latitude'].isna() | stores['longitude'].isna()]
if len(still_missing) == 0:
    print("All coordinates have been added successfully!")
else:
    print(f"Still missing coordinates for {len(still_missing)} stores:")
    print(still_missing[['location', 'state']])

All coordinates have been added successfully!


## Adding the temperature and weather conditions using VisualCrossing.com API

In [ ]:
#Importing required libraries
import pandas as pd
import requests
import time

def get_weather_data(sales_df, api_key):
    base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    
    state_coords = {
        'TX': {'lat': 30.2672, 'lon': -97.7431},  #Austin
        'FL': {'lat': 28.5383, 'lon': -81.3792},  #Orlando
        'CA': {'lat': 34.0522, 'lon': -118.2437}, #LA
        'NC': {'lat': 35.7796, 'lon': -78.6382},  #Raleigh
        'GA': {'lat': 33.7490, 'lon': -84.3880},  #Atlanta
        'KY': {'lat': 38.2527, 'lon': -85.7585},  #Louisville
        'NY': {'lat': 40.7128, 'lon': -74.0060},  #NYC
        'TN': {'lat': 36.1627, 'lon': -86.7816},  #Nashville
        'AR': {'lat': 34.7465, 'lon': -92.2896},  #Little Rock
        'AL': {'lat': 33.5186, 'lon': -86.8104},  #Birmingham
        'UT': {'lat': 40.7608, 'lon': -111.8910}, #Salt Lake City
        'IL': {'lat': 41.8781, 'lon': -87.6298},  #Chicago
        'IN': {'lat': 39.7684, 'lon': -86.1581},  #Indianapolis
        'MO': {'lat': 38.6270, 'lon': -90.1994},  #St. Louis
        'NM': {'lat': 35.6870, 'lon': -105.9378}  #Santa Fe
    }

    #Adding week number and weather columns
    sales_df['week'] = pd.to_datetime(sales_df['date']).dt.isocalendar().week
    sales_df['temperature'] = None
    sales_df['weather_condition'] = None
    
    for state in sales_df['state'].unique():
        print(f"\nGetting weather for {state}")
        
        for week in sorted(sales_df[sales_df['state'] == state]['week'].unique()):
            try:
                
                sample_date = sales_df[
                    (sales_df['state'] == state) & 
                    (sales_df['week'] == week)
                ]['date'].min()
                
                date_str = sample_date.strftime('%Y-%m-%d')
                coords = state_coords[state]
                location = f"{coords['lat']},{coords['lon']}"
                url = f"{base_url}/{location}/{date_str}?unitGroup=metric&key={api_key}"
                
                response = requests.get(url)
                
                if response.status_code == 200:
                    data = response.json()
                    temp = data['days'][0]['temp']
                    condition = data['days'][0]['conditions']
                    
                    mask = (sales_df['state'] == state) & (sales_df['week'] == week)
                    sales_df.loc[mask, 'temperature'] = temp
                    sales_df.loc[mask, 'weather_condition'] = condition
                    
                    print(f"Week {week}: {temp}°C, {condition}")
                else:
                    print(f"Error for week {week}: {response.status_code}")
                
                time.sleep(1)
                
            except Exception as e:
                print(f"Failed for week {week}: {str(e)}")
                continue

    return sales_df.drop('week', axis=1)


if __name__ == "__main__":
    #Loading data
    print("Loading data from sales_data.csv...")
    sales_df = pd.read_csv('sales_data.csv')
    sales_df['date'] = pd.to_datetime(sales_df['date'])
    
    #Adding weather data using the API key
    api_key = 'FW7ULVFY572R8U6CJHF5FZEA8'
    sales_df = get_weather_data(sales_df, api_key)
    
    #Saving back to same file
    sales_df.to_csv('sales_data.csv', index=False)
    print("\nUpdated sales_data_temperature.csv with weather data!")
    

## Adding the temperature and the weather condition to the sale_data.csv

In [9]:
import pandas as pd

#Loading both files
sales_df = pd.read_csv('sales_data.csv')
temp_df = pd.read_csv('sales_data.csv')

#Merging files wrt date and state
sales_df = pd.merge(
    sales_df,
    temp_df[['date', 'state', 'temperature', 'weather_condition']],
    on=['date', 'state']
)

#Saving updated file
sales_df.to_csv('sales_data.csv', index=False)


## Calculating the sales per square foot and adding it as a feature

In [10]:
import pandas as pd
import numpy as np

#Loading store data
store_df = pd.read_csv('store_data.csv')

#Calculating sales per square foot
store_df['sales_per_sqft'] = (store_df['monthly_sales'] * 12) / store_df['size_sqft']

#Rounding to 2 decimal places
store_df['sales_per_sqft'] = store_df['sales_per_sqft'].round(2)

#Saving to store_data.csv
store_df.to_csv('store_data.csv', index=False)

#Printing summary
print("\nSales per Square Foot Summary by Store Category:")
print(store_df.groupby('category')['sales_per_sqft'].agg(['mean', 'min', 'max']))

#The data shows that Neighborhood Markets have the highest sales per 
#square foot (averaging $207/sqft) because they're smaller stores with focused inventory, 
#while Supercenters have lower sales per square foot (averaging $104/sqft) despite higher 
#total sales because they have much larger floor spaces.


Sales per Square Foot Summary by Store Category:
                           mean     min     max
category                                       
Discount Store       119.025000   96.10  134.14
Neighborhood Market  207.193333  120.65  303.25
Supercenter          103.775882   68.04  161.08
